# Séance 1 — Rappel : Analyse de données avec Pandas (Scraping & préprocessing)

## Objectif

Fournir une méthode pratique et reproductible pour passer d’un **jeu de fichiers mensuels bruts** (NYC Yellow Taxi) à un **jeu de données tabulaire propre, enrichi et prêt pour de l’analyse ou du machine learning**. À l’issue de la séance, vous saurez : charger et concaténer des fichiers parquet/CSV, nettoyer les valeurs aberrantes, parser et exploiter des timestamps, créer des features temporelles et géographiques, agréger par zone/heure, et exporter des livrables réutilisables.

## Description des données

Nous utilisons les **fichiers mensuels Yellow Taxi (NYC)** au format Parquet : `yellow_tripdata_{YYYY}-{MM}.parquet`. Le script fourni permet d’itérer sur une plage temporelle (par défaut `2009-01` → `2025-12`) et de récupérer en plus le fichier de référence des zones `taxi_zone_lookup.csv`.
Colonnes courantes (exemples) :

* `tpep_pickup_datetime`, `tpep_dropoff_datetime` (horodatages)
* `PULocationID`, `DOLocationID` (identifiants de zone)
* `trip_distance`, `passenger_count` (mesures)
* `fare_amount`, `tip_amount`, `total_amount`, `payment_type` (tarification)
  Chaque fichier mensuel contient un grand nombre d’enregistrements : pour une session en salle, privilégier un **échantillon** ou un **mois réduit** plutôt que de charger l’intégralité de la plage en mémoire.

## Récupération des fichiers

Le script de téléchargement utilise les URLs publiques du dataset (`yellow_tripdata_{YYYY}-{MM}.parquet`) et télécharge également `taxi_zone_lookup.csv`. Pour la séance, les options recommandées :

* utiliser un **échantillon** (ex. `yellow_tripdata_sample.csv` ou un seul mois), ou
* télécharger 1–3 mois représentatifs (p. ex. janvier / juillet / décembre d’une année).
  Le script crée les dossiers nécessaires et convertit le lookup CSV en parquet pour accélérer les lectures ultérieures.

## Tâches & transformations avec Pandas

Les opérations couvertes et démontrées dans le TP :

1. **Chargement & inspection**

   * `pd.read_parquet` / `pd.read_csv` ; `head()`, `shape`, `dtypes`, comptage NaN.

2. **Concaténation incrémentale**

   * lire fichier par fichier et concaténer (ou streamer / échantillonner) pour éviter d’épuiser la mémoire.

3. **Nettoyage basique**

   * suppression des lignes impossibles ou aberrantes : `trip_distance <= 0`, `fare_amount <= 0`, `trip_duration <= 0`.
   * normalisation des textes/IDs si nécessaire, gestion des valeurs manquantes (`fillna` / `dropna`).

4. **Parsing et gestion des dates**

   * conversion en `datetime` (`pd.to_datetime`), calcul de la durée (`dropoff - pickup`), détection d’erreurs de parsing.
   * extraction de composantes temporelles : `pickup_hour`, `pickup_day`, `pickup_weekday`, `pickup_month`, `is_night`.

5. **Feature engineering tabulaire**

   * `fare_per_km = fare_amount / trip_distance` (avec gestion des divisions par zéro).
   * `tip_ratio = tip_amount / fare_amount`.
   * indicateurs (ex. `is_long_trip`, `is_shared_zone`) et colonnes dérivées (`n_words` style analogies si texte présent).

6. **Jointures / merges**

   * joindre `taxi_zone_lookup` sur `PULocationID`/`DOLocationID` pour obtenir `pickup_borough`, `pickup_zone` et enrichir l’analyse géographique.

7. **Aggregations & groupby**

   * agrégations par zone / heure / jour : `count`, `mean(trip_distance)`, `median(fare_amount)`, `mean(tip_ratio)`, `max(trip_duration)`.
   * création de tables résumées utiles pour visualisations et features d’agrégation.

8. **Opérations ligne-à-ligne**

   * usage ponctuel de `apply` pour transformations personnalisées (avec mise en garde sur la performance : préférer les opérations vectorisées quand c’est possible).

9. **Export**

   * export final en CSV/Parquet (`cleaned_trips.csv` / `cleaned_trips.parquet`) prêt pour modélisation ou ingestion dans un pipeline ML.

## Livrables attendus

* `s1_pandas.ipynb` : notebook bien commenté (Markdown + cellules de code) avec étapes reproductibles.
* `cleaned_trips.csv` ou `cleaned_trips.parquet` : dataset nettoyé et réduit aux colonnes pertinentes pour ML.
* `summary_by_zone.csv` : table résumé par zone (nombre de trajets, distance moyenne, tarif moyen, ratio pourboire).
* Optional : notebook ou script d’échantillonnage si traitement complet trop lourd.

## Bonnes pratiques et remarques opérationnelles

* **Échantillonnage** : pour la classe, limiter le volume (100k–500k lignes) afin de conserver interactivité.
* **Mémoire** : lire fichier par fichier, utiliser `dtype` explicites et `usecols` pour réduire la charge mémoire.
* **Robustesse** : toujours vérifier `dtypes` après lecture et utiliser `indicator=True` lors des merges pour diagnostiquer les clés non appariées.
* **Chunking des textes** : si vous ajoutez des colonnes textuelles volumineuses (ex. notes), découper avant export pour embeddings.
* **Reproductibilité** : versionner le script de téléchargement et enregistrer la liste des fichiers sources (hashs ou timestamps).

# 1 -  Script de téléchargement des données

In [1]:
import requests
import pandas as pd 
import os
NYC_TRIPS_URL = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{}-{}.parquet"
DATSET_FOLDER = 'yellow_tripdata'
TAXI_ZONE_URL = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"

def verify_if_file_already_downloaded(file_path: str) -> bool:
    """
        Verify if file already downloaded
        Args:
            file_path (str): File path
        Returns:
            bool: True if file already downloaded, False otherwise
    """
    return os.path.exists(file_path)

def format_url(year: str, month: str) -> str:
    """
        Format URL
        Args:
            year (str): Year
            month (str): Month
        Returns:
            str: Formatted URL
    """
    return NYC_TRIPS_URL.format(year, month)

def generate_month_range(start_month : str = '2009-01', 
                         end_month : str = '2025-12'
                        ) -> list:
    """
        Generate month range
        Args:
            start_month (str): Start month
            end_month (str): End month
        Returns:
            list: Month range
    """
    start_year, start_month = int(start_month[:4]), int(start_month[5:])
    end_year, end_month = int(end_month[:4]), int(end_month[5:])  # Correction ici: end_month[5:] au lieu de end_month[4:]
    month_range = []
    for year in range(start_year, end_year + 1):
        for month in range(start_month if year == start_year else 1, end_month + 1 if year == end_year else 13):
            month_range.append(f"{year}-{month:02d}")
    return month_range
def download_data(url: str, file_path: str) -> None:
    """
        Download data from URL
        Args:
            url (str): URL
            file_path (str): File path
        Returns:
            None
    """
    print(f"Downloading data from {url} to {file_path}")
    response = requests.get(url)
    # Check if request was successful
    if response.status_code == 200:
        print(f"Data downloaded from {url} with status code {response.status_code}")
    elif response.status_code == 403 : 
        print(f"File {url} not found, status code {response.status_code}")
        raise Exception(f"File {url} not found, status code {response.status_code}")
    else:
        print(f"Failed to download data from {url} with status code {response.status_code}")
        raise Exception(f"Failed to download data from {url} with status code {response.status_code}")
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, "wb") as f:
        f.write(response.content)
    print(f"Data downloaded from {url} to {file_path}")
    return file_path

def download_data_for_month(year: str, month: str, download_dir: str = f"./data/{DATSET_FOLDER}") -> str:
    """
        Download data for month
        Args:
            year (str): Year
            month (str): Month
            download_dir (str): Download directory
        Returns:
            str: File path
    """
    url = format_url(year, month)
    file_path = os.path.join(download_dir, f"{year}-{month}.parquet")
    if verify_if_file_already_downloaded(file_path):
        print(f"File {file_path} already downloaded, skipping download")
        return file_path
    return download_data(url, file_path)

def download_data_month_to_month(
                                start_month : str = '2009-01', 
                                end_month : str = '2025-12', 
                                download_dir: str = f"./data/{DATSET_FOLDER}"
) -> None:
    """
        Download data month to month
        Args:
            start_month (str): Start month
            end_month (str): End month
            download_dir (str): Download directory
        Returns:
            None
    """
    print(f"Downloading data from {start_month} to {end_month} to {download_dir}")
    month_range = generate_month_range(start_month, end_month)
    for month in month_range:
        year, month_num = month.split('-')
        try:
            download_data_for_month(year, month_num, download_dir)
            print(f"Successfully downloaded data for {year}-{month_num} to {download_dir}")
        except Exception as e:
            print(f"Failed to download data for {year}-{month_num}, error: {e}")
    print(f"Downloaded data from {start_month} to {end_month} to {download_dir}")
    
def download_taxi_zones() -> None:
    """
    Télécharge le fichier des zones de taxi, le convertit en parquet et supprime le CSV.
    """
    url = TAXI_ZONE_URL
    csv_path = "./data/yellow_tripdata/taxi_zones.csv"
    parquet_path = "./data/yellow_tripdata/taxi_zones.parquet"
    
    # Création du dossier si nécessaire
    os.makedirs("./data/yellow_tripdata", exist_ok=True)
    
    # Téléchargement du fichier
    print(f"Téléchargement des zones de taxi depuis {url}")
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Échec du téléchargement des zones de taxi. Code: {response.status_code}")
        return
        
    # Sauvegarde du CSV
    with open(csv_path, "wb") as f:
        f.write(response.content)
    print(f"Fichier CSV sauvegardé: {csv_path}")
    
    # Conversion en parquet
    try:
        df = pd.read_csv(csv_path)
        df.to_parquet(parquet_path)
        print(f"Fichier converti en parquet: {parquet_path}")
        
        # Suppression du CSV
        os.remove(csv_path)
        print("Fichier CSV supprimé")
    except Exception as e:
        print(f"Erreur lors de la conversion: {str(e)}")

In [2]:
download_data_month_to_month(start_month='2023-01', end_month='2026-01')

Data downloaded from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet with status code 200
Data downloaded from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet to ./data/yellow_tripdata/2023-01.parquet
Successfully downloaded data for 2023-01 to ./data/yellow_tripdata
Data downloaded from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet with status code 200
Data downloaded from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet to ./data/yellow_tripdata/2023-02.parquet
Successfully downloaded data for 2023-02 to ./data/yellow_tripdata
Data downloaded from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet with status code 200
Data downloaded from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet to ./data/yellow_tripdata/2023-03.parquet
Successfully downloaded data for 2023-03 to ./data/yellow_tripdata
Data

## Concaténer tous les fichiers mensuels en un seul DataFrame Pandas

In [ ]:
import pandas as pd
df_list = []
for file in sorted(os.listdir(f'./data/{DATSET_FOLDER}')):
    if file.endswith('.parquet'):
        df = pd.read_parquet(os.path.join(f'./data/{DATSET_FOLDER}', file))
        df_list.append(df)
        print(f"{file}: {df.shape}")
combined_df = pd.concat(df_list, ignore_index=True)
print(f"Combined DataFrame shape: {combined_df.shape}")

2023-01.parquet: (3066766, 19)
2023-02.parquet: (2913955, 19)
2023-03.parquet: (3403766, 19)
2023-04.parquet: (3288250, 19)
2023-05.parquet: (3513649, 19)
2023-06.parquet: (3307234, 19)
2023-07.parquet: (2907108, 19)
2023-08.parquet: (2824209, 19)
2023-09.parquet: (2846722, 19)
2023-10.parquet: (3522285, 19)
2023-11.parquet: (3339715, 19)
2023-12.parquet: (3376567, 19)
2024-01.parquet: (2964624, 19)
2024-02.parquet: (3007526, 19)
2024-03.parquet: (3582628, 19)
2024-04.parquet: (3514289, 19)
2024-05.parquet: (3723833, 19)
2024-06.parquet: (3539193, 19)
2024-07.parquet: (3076903, 19)
2024-08.parquet: (2979183, 19)
2024-09.parquet: (3633030, 19)
2024-10.parquet: (3833771, 19)
2024-11.parquet: (3646369, 19)
2024-12.parquet: (3668371, 19)
2025-01.parquet: (3475226, 20)
2025-02.parquet: (3577543, 20)
2025-03.parquet: (4145257, 20)
2025-04.parquet: (3970553, 20)
2025-05.parquet: (4591845, 20)
2025-06.parquet: (4322960, 20)
2025-07.parquet: (3898963, 20)
2025-08.parquet: (3574091, 20)
2025-09.

In [ ]:
combined_df.to_parquet('./data/cleaned/combined_yellow_tripdata_2023_2026.parquet')